In [1]:
#dependencies
import numpy as np #matrix math
import tensorflow as tf #ML

In [2]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [3]:
tf.__version__

'1.5.0'

## Input 

In [4]:
import pickle

In [5]:
path = 'movie_line.pickle'

In [6]:
data = pickle.load(open(path,'rb'))

In [7]:
data.head()

0     They do to !
1      I hope so .
2       She okay ?
3       Let's go .
4              Wow
Name: utterance, dtype: object

In [11]:
BOW = ' '.join(data.values.flatten())

In [12]:
import collections
BOW_Count = collections.Counter(BOW)

In [13]:
BOW_Count

Counter({'\t': 873,
         ' ': 4300516,
         '!': 37865,
         '"': 10143,
         '#': 23,
         '$': 180,
         '%': 71,
         '&': 266,
         "'": 216097,
         ')': 60,
         '*': 1214,
         '+': 30,
         ',': 170375,
         '-': 75136,
         '.': 503913,
         '/': 4069,
         '0': 2486,
         '1': 1927,
         '2': 1194,
         '3': 854,
         '4': 809,
         '5': 938,
         '6': 584,
         '7': 622,
         '8': 627,
         '9': 785,
         ':': 1756,
         ';': 1586,
         '<': 7787,
         '=': 16,
         '>': 7786,
         '?': 110240,
         'A': 40594,
         'B': 26691,
         'C': 24115,
         'D': 26186,
         'E': 14534,
         'F': 12962,
         'G': 18165,
         'H': 38513,
         'I': 175189,
         'J': 13746,
         'K': 6222,
         'L': 19375,
         'M': 30971,
         'N': 30209,
         'O': 23539,
         'P': 13531,
         'Q': 749,
         '

## Variable

In [ ]:
PAD = 0
EOS = 1

vocab_size = 10 #number 0~9
input_embedding_size = 20 #character length

encoder_hidden_units = 20
decoder_hidden_units = encoder_hidden_units * 2  
#generally decoder should have same size of encoder, change its size will lead to different result

In [ ]:
#placeholders
encoder_inputs = tf.placeholder(shape=(None,None), dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.placeholder(shape=(None,), dtype = tf.int32, name = 'encoder_inputs_length')
decoder_targets = tf.placeholder(shape=(None,None), dtype = tf.int32, name = 'decoder_targets')

### Embedding

In [ ]:
#embeddings
embeddings = tf.Variable(tf.random_uniform([vocab_size,input_embedding_size], -1.0, 1),dtype = tf.float32)
encoder_emb_inp = tf.nn.embedding_lookup(embeddings, encoder_inputs)

### Encoder 

In [ ]:
#defining encoder
encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(encoder_hidden_units)

In [ ]:
#dynamic rnn, bidirectional
((encoder_fw_outputs,
  encoder_bw_outputs),
 (encoder_fw_final_state, 
  encoder_bw_final_state)) = (
    tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                     cell_bw = encoder_cell,
                     inputs = encoder_emb_inp,
                     sequence_length = encoder_inputs_length,
                     dtype=tf.float32,
                     time_major = True))

In [ ]:
#bidirectional step
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)

encoder_final_state_c = tf. concat(
                        (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
                        (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

encoder_final_state = tf.nn.rnn_cell.LSTMStateTuple(
                      c = encoder_final_state_c,
                      h = encoder_final_state_h)

In [ ]:
encoder_max_time, batch_size = tf.unstack(tf.shape(encoder_inputs))

### Decoder

In [ ]:
#defining decoder
decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(decoder_hidden_units)
decoder_lengths = encoder_inputs_length + 3
# +2 additional steps, +1 leading <EOS> token for decoder inputs

In [ ]:
#output projection
#define our weights and biases

W = tf.Variable(tf.random_uniform([decoder_hidden_units],vocab_size,-1,1), dtype = tf.float32)
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

### Padding 

In [ ]:
#create padded inputs for the decoder from the word embeddings

#telling the program to test a condition, and trigger an error if the condition if false
assert EOS == 1 and PAD == 0

eos_time_slice = tf.ones([batch_size], dtype=tf.int32,name='EOS')
pad_time_slice = tf.zeros([batch_size], dtype=tf.int32, name='PAD')

#retrieves rows of the params tensor. The behavior is similar to using indexing with arrays in numpy
eos_step_embedded = tf.nn.embedding_lookup(embeddings,eos_time_slice)
pad_step_embedded = tf.nn.embedding_lookup(embeddings,pad_time_slice)



In [ ]:
# manually specifying loop function through time - to get intial cell state and input to RNN
# normally we'd just use dynamic_rnn

def loop_fn_initial():
    initial_elements_finished = (0>= decoder_lengths) #all False at the initial step
    #end of sentence
    initial_input = eos_step_embedded
    #last time steps cell state
    initial_cell_state = encoder_final_state
    #none
    initial_cell_output = None
    #none
    initial_loop_state = None
    return (initial_elements_finished,
           initial_input,
           initial_cell_state,
           initial_loop_state)


In [ ]:
#"soft" attention mechanism choose which previously generated token to pass as input in the next time
def loop_fn_transition(time, previous_output, previous_state, previous_loop_state):
    
    def get_next_input():
        output_logits = tf.add(tf.matmul(previous_output,W),b)
        #This is Attention
        prediction = tf.argmax(output_logits, axis=1)
        next_input = tf.nn.embedding_lookup(embeddings,prediction)
        return next_input
    
    elements_finished = (time >= decoder_lengths)
    
    finished = tf.reduce_all(elements_finished)
    input_ = tf.cond(finished, lambda:pad_step_embedded, get_next_input)
    
    #set previous to current
    state = previous_state
    output = previous_output
    loop_state = None
    
    return(elements_finished,
          input_,
          state,
          output,
          loop_state)

In [ ]:
def loop_fn(time,previous_output,previous_state, previous_loop_state):
    if previous_state is None: #time == 0
        assert previous_output is None and previous_state is None
        return loop_fn_initial()
    else:
        return loop_fn_transition(time, previous_output, previous_state, previous_loop_state)

In [ ]:
decoder_outputs_ta, decoder_final_state, _ = tf.nn.raw_rnn(decoder_cell,loop_fn)
decoder_outputs = decoder_outputs_ta.stack()